<a href="https://colab.research.google.com/github/mike-jansen/misinformation-analysis-capstone/blob/main/dataset_creation_for_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import pandas as pd
import numpy as np

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#Preprocessing

In [ ]:
# loading data
path = '/content/drive/MyDrive/output/'
liar_false = pd.read_csv(path + 'liar_false.csv')
liar_true = pd.read_csv(path + 'liar_true.csv')
nela_false = pd.read_csv(path + 'nela_false.csv')
nela_true = pd.read_csv(path + 'nela_true.csv')

In [ ]:
nela_true.rename(columns={'content': 'text'}, inplace=True)
nela_false.rename(columns={'content': 'text'}, inplace=True)

In [ ]:
liar_false = liar_false.head(10000)
liar_true = liar_true = liar_false.head(10000)
nela_false = nela_false.head(10000)
nela_true = nela_true.head(10000)

# Creating the dataset

In [ ]:
!pip install NRCLex
from nrclex import NRCLex
# import the punkt tokenizer for NRCLex to work
import nltk
nltk.download('punkt')
from collections import defaultdict
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43308 sha256=f867dfee6b0450622c8866af41e53498f20b043f0dccc5b6a17c606436c77b0f
  Stored in directory: /root/.cache/pip/wheels/d2/10/44/6abfb1234298806a145fd6bcaec8cbc712e88dd1cd6cb242fa
Successfully built NRCLex


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# function that calculates the NRCLex scores for a piece of text
def calculate_nrc_scores(text):
  if isinstance(text, str):
    lex = NRCLex(text)
    return lex.affect_frequencies

In [ ]:
dataframes = [liar_false, liar_true, nela_false, nela_true]
updated_dataframes = []

# append the NRCLex scores to original datasets
for df in dataframes:
    all_scores = df['text'].apply(calculate_nrc_scores)
    all_scores = all_scores.apply(lambda x: {} if x is None else x)  # Replace None values with {}
    df_new = pd.DataFrame(all_scores.tolist())
    updated_df = pd.concat([df, df_new], axis=1)
    updated_dataframes.append(updated_df)

In [ ]:
liar_false = updated_dataframes[0]
liar_true = updated_dataframes[1]
nela_false = updated_dataframes[2]
nela_true = updated_dataframes[3]

In [ ]:
# export the new dataframes, then use the new features in the decision tree
import os
output_dir = '/content/drive/MyDrive/output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

liar_false.to_csv(os.path.join(output_dir, 'liar_false.csv'), index=False)
liar_true.to_csv(os.path.join(output_dir, 'liar_true.csv'), index=False)
nela_false.to_csv(os.path.join(output_dir, 'nela_false.csv'), index=False)
nela_true.to_csv(os.path.join(output_dir, 'nela_true.csv'), index=False)